In [1]:
%load_ext IPython.extensions.autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd

import openpyxl

import sys
import os
import runpy

import requests
from io import StringIO
import json

import json
import glob
from pathlib import Path

In [3]:
os.chdir('../..')
os.getcwd()


'c:\\Users\\jarem\\OneDrive - London School of Economics\\YEAR 2\\PP4V8 - policy paper\\policy-paper-repo'

# Operationalisation of variables

## Dane GUS

### Programme period: 2003-2007

In [81]:
PL_projects_07_13 = pd.read_excel(r"data\inputs\3. treatment\dane_gov_pl\FE_2007_2013_bez_EFFR.xlsx", header = 0)

In [82]:
PL_projects_07_13.shape

(2886, 87)

#### rename and remove columns

In [83]:
cols_to_keep = [ 'Kod' , 'Nazwa',
        'wartość całkowita projektów zakończonych;ogółem;dofinansowanie ze środków Unii Europejskiej (wydatki kwalifikowalne);31 grudnia;2011;[zł]',
        'wartość całkowita projektów zakończonych;ogółem;dofinansowanie ze środków Unii Europejskiej (wydatki kwalifikowalne);31 grudnia;2012;[zł]',
        'wartość całkowita projektów zakończonych;ogółem;dofinansowanie ze środków Unii Europejskiej (wydatki kwalifikowalne);31 grudnia;2013;[zł]',
        'wartość całkowita projektów zakończonych;ogółem;dofinansowanie ze środków Unii Europejskiej (wydatki kwalifikowalne);31 grudnia;2014;[zł]',
        'wartość całkowita projektów zakończonych;ogółem;dofinansowanie ze środków Unii Europejskiej (wydatki kwalifikowalne);31 grudnia;2015;[zł]',
        'wartość całkowita projektów zakończonych;ogółem;dofinansowanie ze środków Unii Europejskiej (wydatki kwalifikowalne);31 grudnia;2016;[zł]',
        'wartość całkowita podpisanych umów o dofinansowanie;ogółem;wartość ogółem (wydatki kwalifikowalne + niekwalifikowalne);31 grudnia;2011;[zł]',
        'wartość całkowita podpisanych umów o dofinansowanie;ogółem;wartość ogółem (wydatki kwalifikowalne + niekwalifikowalne);31 grudnia;2012;[zł]',
        'wartość całkowita podpisanych umów o dofinansowanie;ogółem;wartość ogółem (wydatki kwalifikowalne + niekwalifikowalne);31 grudnia;2013;[zł]',
        'wartość całkowita podpisanych umów o dofinansowanie;ogółem;wartość ogółem (wydatki kwalifikowalne + niekwalifikowalne);31 grudnia;2014;[zł]',
        'wartość całkowita podpisanych umów o dofinansowanie;ogółem;wartość ogółem (wydatki kwalifikowalne + niekwalifikowalne);31 grudnia;2015;[zł]',
        'wartość całkowita podpisanych umów o dofinansowanie;ogółem;wartość ogółem (wydatki kwalifikowalne + niekwalifikowalne);31 grudnia;2016;[zł]',
        'wartość całkowita podpisanych umów o dofinansowanie;ogółem;wydatki kwalifikowalne ogółem;31 grudnia;2011;[zł]',
        'wartość całkowita podpisanych umów o dofinansowanie;ogółem;wydatki kwalifikowalne ogółem;31 grudnia;2012;[zł]',
        'wartość całkowita podpisanych umów o dofinansowanie;ogółem;wydatki kwalifikowalne ogółem;31 grudnia;2013;[zł]',
        'wartość całkowita podpisanych umów o dofinansowanie;ogółem;wydatki kwalifikowalne ogółem;31 grudnia;2014;[zł]',
        'wartość całkowita podpisanych umów o dofinansowanie;ogółem;wydatki kwalifikowalne ogółem;31 grudnia;2015;[zł]',
        'wartość całkowita podpisanych umów o dofinansowanie;ogółem;wydatki kwalifikowalne ogółem;31 grudnia;2016;[zł]',
        'wartość całkowita podpisanych umów o dofinansowanie;ogółem;dofinansowanie ze środków Unii Europejskiej (wydatki kwalifikowalne);31 grudnia;2011;[zł]',
        'wartość całkowita podpisanych umów o dofinansowanie;ogółem;dofinansowanie ze środków Unii Europejskiej (wydatki kwalifikowalne);31 grudnia;2012;[zł]',
        'wartość całkowita podpisanych umów o dofinansowanie;ogółem;dofinansowanie ze środków Unii Europejskiej (wydatki kwalifikowalne);31 grudnia;2013;[zł]',
        'wartość całkowita podpisanych umów o dofinansowanie;ogółem;dofinansowanie ze środków Unii Europejskiej (wydatki kwalifikowalne);31 grudnia;2014;[zł]',
        'wartość całkowita podpisanych umów o dofinansowanie;ogółem;dofinansowanie ze środków Unii Europejskiej (wydatki kwalifikowalne);31 grudnia;2015;[zł]',
        'wartość całkowita podpisanych umów o dofinansowanie;ogółem;dofinansowanie ze środków Unii Europejskiej (wydatki kwalifikowalne);31 grudnia;2016;[zł]',
        'wartość całkowita projektów zakończonych;ogółem;wydatki kwalifikowalne ogółem;31 grudnia;2011;[zł]',
        'wartość całkowita projektów zakończonych;ogółem;wydatki kwalifikowalne ogółem;31 grudnia;2012;[zł]',
        'wartość całkowita projektów zakończonych;ogółem;wydatki kwalifikowalne ogółem;31 grudnia;2013;[zł]',
        'wartość całkowita projektów zakończonych;ogółem;wydatki kwalifikowalne ogółem;31 grudnia;2014;[zł]',
        'wartość całkowita projektów zakończonych;ogółem;wydatki kwalifikowalne ogółem;31 grudnia;2015;[zł]',
        'wartość całkowita projektów zakończonych;ogółem;wydatki kwalifikowalne ogółem;31 grudnia;2016;[zł]',
]

In [84]:
PL_projects_07_13 = PL_projects_07_13[cols_to_keep]

In [85]:
cols_rename_dict = { 'Kod' : 'code' , 'Nazwa' : 'name',
        'wartość całkowita projektów zakończonych;ogółem;dofinansowanie ze środków Unii Europejskiej (wydatki kwalifikowalne);31 grudnia;2011;[zł]' : 'total_eligible_value_completed_projects_EU_funding_2011_PLN',
        'wartość całkowita projektów zakończonych;ogółem;dofinansowanie ze środków Unii Europejskiej (wydatki kwalifikowalne);31 grudnia;2012;[zł]' : 'total_eligible_value_completed_projects_EU_funding_2012_PLN',
        'wartość całkowita projektów zakończonych;ogółem;dofinansowanie ze środków Unii Europejskiej (wydatki kwalifikowalne);31 grudnia;2013;[zł]' : 'total_eligible_value_completed_projects_EU_funding_2013_PLN',
        'wartość całkowita projektów zakończonych;ogółem;dofinansowanie ze środków Unii Europejskiej (wydatki kwalifikowalne);31 grudnia;2014;[zł]' : 'total_eligible_value_completed_projects_EU_funding_2014_PLN',
        'wartość całkowita projektów zakończonych;ogółem;dofinansowanie ze środków Unii Europejskiej (wydatki kwalifikowalne);31 grudnia;2015;[zł]' : 'total_eligible_value_completed_projects_EU_funding_2015_PLN',
        'wartość całkowita projektów zakończonych;ogółem;dofinansowanie ze środków Unii Europejskiej (wydatki kwalifikowalne);31 grudnia;2016;[zł]' : 'total_eligible_value_completed_projects_EU_funding_2016_PLN',
        'wartość całkowita podpisanych umów o dofinansowanie;ogółem;wartość ogółem (wydatki kwalifikowalne + niekwalifikowalne);31 grudnia;2011;[zł]' : 'total_value_signed_financing_agreements_total_value_2011_PLN',
        'wartość całkowita podpisanych umów o dofinansowanie;ogółem;wartość ogółem (wydatki kwalifikowalne + niekwalifikowalne);31 grudnia;2012;[zł]' : 'total_value_signed_financing_agreements_total_value_2012_PLN',
        'wartość całkowita podpisanych umów o dofinansowanie;ogółem;wartość ogółem (wydatki kwalifikowalne + niekwalifikowalne);31 grudnia;2013;[zł]' : 'total_value_signed_financing_agreements_total_value_2013_PLN',
        'wartość całkowita podpisanych umów o dofinansowanie;ogółem;wartość ogółem (wydatki kwalifikowalne + niekwalifikowalne);31 grudnia;2014;[zł]' : 'total_value_signed_financing_agreements_total_value_2014_PLN',
        'wartość całkowita podpisanych umów o dofinansowanie;ogółem;wartość ogółem (wydatki kwalifikowalne + niekwalifikowalne);31 grudnia;2015;[zł]' : 'total_value_signed_financing_agreements_total_value_2015_PLN',
        'wartość całkowita podpisanych umów o dofinansowanie;ogółem;wartość ogółem (wydatki kwalifikowalne + niekwalifikowalne);31 grudnia;2016;[zł]' : 'total_value_signed_financing_agreements_total_value_2016_PLN',
        'wartość całkowita podpisanych umów o dofinansowanie;ogółem;wydatki kwalifikowalne ogółem;31 grudnia;2011;[zł]' : 'total_eligible_value_signed_financing_agreements_2011_PLN',
        'wartość całkowita podpisanych umów o dofinansowanie;ogółem;wydatki kwalifikowalne ogółem;31 grudnia;2012;[zł]' : 'total_eligible_value_signed_financing_agreements_2012_PLN',
        'wartość całkowita podpisanych umów o dofinansowanie;ogółem;wydatki kwalifikowalne ogółem;31 grudnia;2013;[zł]': 'total_eligible_value_signed_financing_agreements_2013_PLN',
        'wartość całkowita podpisanych umów o dofinansowanie;ogółem;wydatki kwalifikowalne ogółem;31 grudnia;2014;[zł]' : 'total_eligible_value_signed_financing_agreements_2014_PLN',
        'wartość całkowita podpisanych umów o dofinansowanie;ogółem;wydatki kwalifikowalne ogółem;31 grudnia;2015;[zł]': 'total_eligible_value_signed_financing_agreements_2015_PLN',
        'wartość całkowita podpisanych umów o dofinansowanie;ogółem;wydatki kwalifikowalne ogółem;31 grudnia;2016;[zł]' : 'total_eligible_value_signed_financing_agreements_2016_PLN',
        'wartość całkowita podpisanych umów o dofinansowanie;ogółem;dofinansowanie ze środków Unii Europejskiej (wydatki kwalifikowalne);31 grudnia;2011;[zł]' : 'total_eligible_value_signed_financing_agreements_EU_funding_2011_PLN',
        'wartość całkowita podpisanych umów o dofinansowanie;ogółem;dofinansowanie ze środków Unii Europejskiej (wydatki kwalifikowalne);31 grudnia;2012;[zł]' : 'total_eligible_value_signed_financing_agreements_EU_funding_2012_PLN',
        'wartość całkowita podpisanych umów o dofinansowanie;ogółem;dofinansowanie ze środków Unii Europejskiej (wydatki kwalifikowalne);31 grudnia;2013;[zł]' : 'total_eligible_value_signed_financing_agreements_EU_funding_2013_PLN',
        'wartość całkowita podpisanych umów o dofinansowanie;ogółem;dofinansowanie ze środków Unii Europejskiej (wydatki kwalifikowalne);31 grudnia;2014;[zł]': 'total_eligible_value_signed_financing_agreements_EU_funding_2014_PLN',
        'wartość całkowita podpisanych umów o dofinansowanie;ogółem;dofinansowanie ze środków Unii Europejskiej (wydatki kwalifikowalne);31 grudnia;2015;[zł]': 'total_eligible_value_signed_financing_agreements_EU_funding_2015_PLN',
        'wartość całkowita podpisanych umów o dofinansowanie;ogółem;dofinansowanie ze środków Unii Europejskiej (wydatki kwalifikowalne);31 grudnia;2016;[zł]' : 'total_eligible_value_signed_financing_agreements_EU_funding_2016_PLN',
        'wartość całkowita projektów zakończonych;ogółem;wydatki kwalifikowalne ogółem;31 grudnia;2011;[zł]' : 'total_eligible_value_completed_projects_2011_PLN',
        'wartość całkowita projektów zakończonych;ogółem;wydatki kwalifikowalne ogółem;31 grudnia;2012;[zł]' : 'total_eligible_value_completed_projects_2012_PLN',
        'wartość całkowita projektów zakończonych;ogółem;wydatki kwalifikowalne ogółem;31 grudnia;2013;[zł]' : 'total_eligible_value_completed_projects_2013_PLN',
        'wartość całkowita projektów zakończonych;ogółem;wydatki kwalifikowalne ogółem;31 grudnia;2014;[zł]' : 'total_eligible_value_completed_projects_2014_PLN',
        'wartość całkowita projektów zakończonych;ogółem;wydatki kwalifikowalne ogółem;31 grudnia;2015;[zł]': 'total_eligible_value_completed_projects_2015_PLN',
        'wartość całkowita projektów zakończonych;ogółem;wydatki kwalifikowalne ogółem;31 grudnia;2016;[zł]' : 'total_eligible_value_completed_projects_2016_PLN',
}

In [86]:
PL_projects_07_13.rename(columns=cols_rename_dict, inplace=True)

In [87]:
PL_projects_07_13.head(3)

,code,name,total_eligible_value_completed_projects_EU_funding_2011_PLN,total_eligible_value_completed_projects_EU_funding_2012_PLN,total_eligible_value_completed_projects_EU_funding_2013_PLN,total_eligible_value_completed_projects_EU_funding_2014_PLN,total_eligible_value_completed_projects_EU_funding_2015_PLN,total_eligible_value_completed_projects_EU_funding_2016_PLN,total_value_signed_financing_agreements_total_value_2011_PLN,total_value_signed_financing_agreements_total_value_2012_PLN,...,total_eligible_value_signed_financing_agreements_EU_funding_2013_PLN,total_eligible_value_signed_financing_agreements_EU_funding_2014_PLN,total_eligible_value_signed_financing_agreements_EU_funding_2015_PLN,total_eligible_value_signed_financing_agreements_EU_funding_2016_PLN,total_eligible_value_completed_projects_2011_PLN,total_eligible_value_completed_projects_2012_PLN,total_eligible_value_completed_projects_2013_PLN,total_eligible_value_completed_projects_2014_PLN,total_eligible_value_completed_projects_2015_PLN,total_eligible_value_completed_projects_2016_PLN
0,0,POLSKA,2.129330e+10,3.928128e+10,6.300138e+10,9.270788e+10,1.514780e+11,NaN,3.488748e+11,4.185678e+11,...,2.676541e+11,2.854900e+11,2.899497e+11,NaN,3.311223e+10,6.167440e+10,9.891130e+10,1.416249e+11,2.214065e+11,NaN
1,200000,DOLNOŚLĄSKIE,1.151720e+09,2.258852e+09,4.026069e+09,8.525640e+09,1.189545e+10,NaN,2.295337e+10,2.893788e+10,...,1.860232e+10,1.948053e+10,1.967197e+10,NaN,1.787704e+09,3.650780e+09,6.678295e+09,1.288829e+10,1.809863e+10,NaN
2,201000,Powiat bolesławiecki,3.523231e+07,4.352682e+07,8.424246e+07,1.105440e+08,1.688417e+08,NaN,3.678199e+08,5.574197e+08,...,3.337926e+08,3.486724e+08,2.863434e+08,NaN,5.335319e+07,7.196908e+07,1.709898e+08,2.119871e+08,2.988681e+08,NaN


In [90]:
# 1. IDENTIFY ID COLUMNS
df = PL_projects_07_13.copy()
id_cols = ['code', 'name'] 

# 2. MELT THE DATAFRAME -->  converts columns to rows
df_melted = df.melt(id_vars=id_cols, var_name='raw_column', value_name='Value_PLN')

# 3. EXTRACT VARIABLE AND YEAR
# We use regex to capture: (Everything before the year) _ (The Year) _ PLN
pattern = r'^(.*)_(\d{4})_PLN$'

extracted_data = df_melted['raw_column'].str.extract(pattern)
df_melted['variable'] = extracted_data[0] # The long variable name
df_melted['Year'] = extracted_data[1]   # The Year (2011, 2012...)

# 4. PIVOT TO DESIRED STRUCTURE
df_final = df_melted.pivot_table(
    index=id_cols + ['variable'], 
    columns='Year', 
    values='Value_PLN',
    aggfunc='sum' # Use sum just in case there are duplicates, usually not needed
).reset_index()

# Optional: Clean up the column names (remove the 'Year' label)
df_final.columns.name = None

In [95]:
df_final['EUR_PL_rate'] = 4.4975

In [107]:
# 1. Define the columns that contain the values to convert
year_cols = ['2011', '2012', '2013', '2014', '2015', '2016']
exchange_rate = 4.4975

# 2. Create a copy of the dataframe so we don't overwrite the PLN data
df_eur = df_final.copy()

# 3. Perform the conversion (Division)
df_eur[year_cols] = df_eur[year_cols] / exchange_rate

# 4. Update the 'variable' column name to reflect the change
df_eur['variable'] = df_eur['variable'] + '_EUR'

df_combined = pd.concat([df_final, df_eur], ignore_index=True)
df_combined = df_combined.sort_values(by=['code', 'variable'])


In [ ]:
df_combined.to_parquet(r'data/clean/treatment/eu_flows/gmina_FE_2007_13_pln_eur.parquet')

,code,name,variable,2011,2012,2013,2014,2015,2016,EUR_PL_rate
0,0,POLSKA,total_eligible_value_completed_projects,3.311223e+10,6.167440e+10,9.891130e+10,1.416249e+11,2.214065e+11,0.0,4.4975
14430,0,POLSKA,total_eligible_value_completed_projects_EUR,7.362363e+09,1.371304e+10,2.199251e+10,3.148970e+10,4.922879e+10,0.0,4.4975
1,0,POLSKA,total_eligible_value_completed_projects_EU_fun...,2.129330e+10,3.928128e+10,6.300138e+10,9.270788e+10,1.514780e+11,0.0,4.4975
14431,0,POLSKA,total_eligible_value_completed_projects_EU_fun...,4.734474e+09,8.734025e+09,1.400809e+10,2.061320e+10,3.368050e+10,0.0,4.4975
2,0,POLSKA,total_eligible_value_signed_financing_agreements,2.817125e+11,3.396549e+11,3.882321e+11,4.085475e+11,4.075419e+11,0.0,4.4975
...,...,...,...,...,...,...,...,...,...,...
28857,3263011,Świnoujście (1),total_eligible_value_signed_financing_agreemen...,2.341874e+08,3.541099e+08,4.300761e+08,4.797162e+08,4.980905e+08,0.0,4.4975
14428,3263011,Świnoujście (1),total_eligible_value_signed_financing_agreemen...,6.477136e+08,1.056532e+09,1.253194e+09,1.376496e+09,1.429976e+09,0.0,4.4975
28858,3263011,Świnoujście (1),total_eligible_value_signed_financing_agreemen...,1.440164e+08,2.349153e+08,2.786424e+08,3.060579e+08,3.179491e+08,0.0,4.4975
14429,3263011,Świnoujście (1),total_value_signed_financing_agreements_total_...,3.802724e+09,4.132632e+09,4.448612e+09,4.449113e+09,4.486378e+09,0.0,4.4975


### Programme period: 2014 - 2020

In [41]:
projects_14_20 = pd.read_excel(r"data\inputs\dane_gus\lista_projektow_UE_2014_20\Lista_projektow_FE_2014_2020_02112025.xlsx", header = 2, decimal=',')

In [43]:
df = projects_14_20.copy()

#### Clean up data

In [44]:
from src.functions import *

In [45]:
df.drop(columns=[
    "Tytuł projektu/ Project name",
    "Skrócony opis [do 2000 znaków] / Project summary",
    "Forma finansowania/ Form of finance",
    "Projekt realizowany w ramach terytorialnych mechanizmów wdrażania/ Project implemented under territorial delivery mechanisms",
    "Działanie/ Measure",
    "Poddziałanie/ Submeasure",
    "Dziedzina działalności gospodarczej, której dotyczy projekt/ Area of economic activity",
    "Obszar wsparcia projektu/ Area of project intervention",
    "Cel projektu/ Project thematic objective",
    "Cel uzupełniający dla projektów EFS/ ESF secondary theme"
], inplace=True)
df = clean_english_colnames(df, normalize='keep')   
df = df[df['Fund'] != 'BAR'] 

In [47]:
df.shape

(98553, 15)

### Programme period 2021-2027

In [82]:
projects_21_27 = pd.read_excel(r"data\inputs\dane_gus\lista_projektow_UE_2021_27\Lista_projektow_FE_2021_2027_02112025.xlsx", header = 1, decimal=',')
df_new = projects_21_27.copy()
#### Clean up 
df_new.drop(columns=[
    "Działanie/ Measure",
    # "Cel szczegółowy/ Specific objective",
    "Priorytet/ Priority", 
    "Działanie/ Measure",
    "Numer identyfikacyjny statku w rejestrze floty rybackiej UE/ The Union fishing fleet register identification number",
    "Nazwa wykonawcy kontraktu/ The contractor’s name",
    "Kategoria wsparcia/ Type of intervention"
], inplace=True)

df_new = clean_english_colnames(df_new, normalize='keep')   
df_new = df_new[df_new['Fund'] != 'BAR'] 

df_new.drop(columns = ['Project name', 'Project summary', 'Specific objective',  'EURO exchange rate'], inplace=True)
# 4.4975 # this exchange rate was taken from the report
df_new.columns

Index(['Contract number', 'Beneficiary name', 'Fund', 'Programme',
       'Total project value (PLN)', 'Union co-financing rate (%)',
       'EU co-financing (PLN)', 'Project location', 'Project start date',
       'Project end date'],
      dtype='object')

### Concat 2014-20 and 2021-2027

#### unify colnames across all datasets

In [86]:
df_new.rename(columns={
    
    'Total project value (PLN)': 'Total project value (PLN, for ETC projects EUR)',
    'EU co-financing (PLN)': 'Amount of EU co-financing (PLN, for ETC projects EUR)', 
}, inplace=True)

In [ ]:
df.drop(columns = [
    'Priority axis',  
    'Project implemented under competitive or non-competitive procedure',
    'Funding completed'
], inplace=True)

In [87]:
df_combined = pd.concat([df, df_new], ignore_index=True)

In [96]:
df_combined = df_combined[df_combined['Project location'] != 'Cały Kraj'] 


### Create a final panel

In [102]:
df_cleaned = clean_extract_and_filter_locations(df_combined, location_col='Project location')


In [103]:
# drop additional columns, rename
df_cleaned.drop(columns=['Programme'], inplace=True)


In [104]:
df_cleaned['Territory type'].unique()

array(['03 Obszary wiejskie (o małej gęstości zaludnienia)',
       '02 Małe obszary miejskie (o ludności >5 000 i średniej gęstości zaludnienia)',
       '01 Duże obszary miejskie (o ludności >50 000 i dużej gęstości zaludnienia)',
       '07 Nie dotyczy', '06 Współpraca ponadnarodowa w ramach EFS',
       '05 Współpraca pomiędzy różnymi obszarami objętymi programami krajowymi lub regionalnymi w kontekście krajowym',
       '04 Obszary współpracy makroregionalnej', nan], dtype=object)

In [105]:
powiat_treatment_matrix = create_treatment_matrix(df_cleaned, geo_level='powiat')


In [106]:
powiat_panel_treatment_matrix = create_panel_treatment_matrix(
    df_cleaned,
    start_date_col='Project start date', 
    end_date_col='Project end date'
    )

In [107]:
final_panel_df = flatten_and_rename_panel(powiat_panel_treatment_matrix)

In [111]:
final_panel_df

,powiat,year,total_project_value,total_eligible_expenditure,cofinancing_value,wojewodztwo,number_of_projects,eu_cofinancing_rate
0,aleksandrowski,2015,4.911591e+06,4.861797e+06,4.254339e+06,KUJAWSKO-POMORSKIE,4,87.749207
1,aleksandrowski,2016,1.175017e+07,1.134022e+07,9.289036e+06,KUJAWSKO-POMORSKIE,20,83.375223
2,aleksandrowski,2017,2.748908e+07,2.440334e+07,1.943342e+07,KUJAWSKO-POMORSKIE,45,81.667986
3,aleksandrowski,2018,4.135340e+07,3.624878e+07,2.866181e+07,KUJAWSKO-POMORSKIE,80,81.334345
4,aleksandrowski,2019,5.856736e+07,5.263531e+07,4.173304e+07,KUJAWSKO-POMORSKIE,97,81.108442
...,...,...,...,...,...,...,...,...
5950,zywiecki,2025,1.834452e+08,1.007877e+05,1.420978e+08,ŚLĄSKIE,186,72.879408
5951,zywiecki,2026,1.686218e+08,0.000000e+00,1.316775e+08,ŚLĄSKIE,169,75.438915
5952,zywiecki,2027,1.026303e+08,0.000000e+00,7.991829e+07,ŚLĄSKIE,62,71.644797
5953,zywiecki,2028,5.323936e+07,0.000000e+00,4.510025e+07,ŚLĄSKIE,12,68.488884


In [109]:
final_panel_df.to_csv(r'data/clean/treatment/eu_flows/powiat_panel_year_2014_27.csv')

In [ ]:
powiat_treatment_matrix.wojewodztwo.unique().tolist()

['MAZOWIECKIE',
 'MAŁOPOLSKIE',
 'ŁÓDZKIE',
 'DOLNOŚLĄSKIE',
 'WIELKOPOLSKIE',
 'LUBELSKIE',
 'POMORSKIE',
 'PODKARPACKIE',
 'PODLASKIE',
 'ZACHODNIOPOMORSKIE',
 'KUJAWSKO-POMORSKIE',
 'WARMIŃSKO-MAZURSKIE',
 'ŚLĄSKIE',
 'ŚWIĘTOKRZYSKIE',
 'LUBUSKIE',
 'OPOLSKIE']